<font face="Courier New"><head><h1><b>Projeto 2<br>Aprendizado de máquina: reconhecimento de números</b></h1></head><br>
<body>
<h1>Introdução</h1><br>
<font size="3">
A grosso modo, Machine Learning, ou aprendizado de máquina, é uma importante área da inteligência artificial onde é possível criar algoritmos para ensinar uma máquina a desempenhar tarefas que automatizam modelos analíticos.

Pega-se um conjunto de dados de entrada e com base em determinados padrões encontrados gera-se saídas das quais podem ser usadas para diversos fins. Muitas de nossas atividades diárias são alimentadas por algoritmos de machine learning que podem incluir:

* Detecção de fraudes.
* Resultados de pesquisa na Web.
* Anúncios em tempo real, tanto em páginas da web como em dispositivos móveis.
* Pontuação de crédito e melhores ofertas.
* Previsão de falhas em equipamento.
* Detecção de invasão em uma determinada rede.
* Reconhecimento de determinados padrões e imagens.

<h1>Conceitos</h1><br>
As redes neurais artificiais são baseadas nas tentativas de simular o sistema nervoso humano no que se refere a sua capacidade de aprender e de se adaptar, em outras palavras, uma simulação muito simplificada da operação do cérebro humano. Estas redes neurais artificiais são constituídas de neurônios artificiais (Figura 1).

<img src="neuron.png" align="center"></img>

<font size="2"><center>Figura 1 - Diagramação de um neurônio artificial.</center></font>

A estrutura de um neurônio pode ser representada como uma composição das seguintes unidades:
* Entradas: <b>Xi,...,Xn</b>;

* Pesos: <b>Wi,...,Wn</b>;

* Função de transferência: <b>$\sum$</b>;

* Entrada de rede de um neurônio: <b>NET</b>;

* Função de ativação: <b>F(x)</b>;

* Saída: <b>Out.</b>

Existem alguns métodos para que a máquina aprenda a reconhecer padrões:

* <b>Aprendizado supervisionado:</b> que consiste em exemplos rotulados. O algoritmo de aprendizagem recebe um conjunto de entradas junto com as saídas corretas correspondentes, e o algoritmo aprende comparando a saída real com as saídas corretas para encontrar erros. Em seguida, ele modifica o modelo de acordo com o que foi aprendido.

* <b>Aprendizado não supervisionado:</b> é usado em dados que não possuem uma saída certa correspondente. O algoritmo deve descobrir o que está sendo mostrado e tem por objetivo explorar os dados e assim encontrar alguma estrutura neles. O aprendizado não supervisionado funciona bem em dados transacionais.

* <b>Aprendizado semisupervisionado:</b> geralmente é usado para as mesmas aplicações que o aprendizado supervisionado, porém ele pode usar tanto dados rotulados quanto não rotulados para o treinamento. Normalmente, utiliza-se uma pequena quantidade de dados rotulados com uma grande quantidade de dados não rotulados. Esse tipo de aprendizagem pode ser usado com métodos como a classificação, regressão e previsão. Os primeiros exemplos disso incluem a identificação do rosto de uma pessoa em uma webcam.

* <b>Aprendizado por reforço:</b> essa opção é muitas vezes usado para a robótica, jogos e navegação. Com o aprendizado por reforço, o algoritmo descobre por meio de tentativa e erro quais ações geram as melhores recompensas.


</font></body></font>

In [5]:
# -*- coding: utf-8 -*-
"""
Created on 10/07/2019 - 13:57:43
@author: Hamilton S. Gama Filho e Hugo B. S. Borges
"""

from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras import backend as K
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Dropout
import numpy as np
import cv2
import sys
import os
import time

#-------------------------------------------------------------------------------------------------------------------------
def img_to_array(img, image_data_format='default'):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.bitwise_not(img)

    if image_data_format == "default":
        image_data_format = K.image_data_format()

    if image_data_format not in ['channels_first', 'channels_last']:
        raise Exception('Formato da imagem desconhecido: ', image_data_format)

    x = np.asarray(img, dtype=K.floatx())

    # imagem colorida
    # (channel, height, width)
    if len(x.shape) == 3:
        if image_data_format == 'channels_first':
            x = x.transpose(2, 0, 1)

    # grayscale
    elif len(x.shape) == 2:
        # already for th format
        x = np.expand_dims(x, axis=0)
        if image_data_format == 'channels_last':
            #(height, channel, width)
            x = x.reshape((x.shape[1], x.shape[2], x.shape[0]))
            #print("CHANNEL")
            #print(x.shape[2])
    # quando o tipo for desconhecido
    else:
        raise Exception('Formato da imagem não suportado: ', x.shape)

    return x
#-------------------------------------------------------------------------------------------------------------------------
def base_model():
    model = Sequential()
    # add model layers
    model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model
#-------------------------------------------------------------------------------------------------------------------------

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("Número de classes")
print(np.unique(y_train))

x_train = x_train.reshape(60000,28, 28, 1)
x_test = x_test.reshape(10000,28,28, 1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

global num_pixels
num_pixels = x_train.shape[1] * x_train.shape[2]

model_name = str(sys.argv[0]).split(".")[0]+".h5" # cria o modelo com o nome do script python.

batch_size  = 32 
num_classes = 10
epochz      = 6 # Nao precisa mais que 1 epoch para o MNIST(treino). Evite overfitting.

global density
density     = 128

#se não tem argumento para o programa, faz o treino e salva o modelo com o nome do script python.
if len(sys.argv) == 1:
    cnn_n = base_model()
    cnn_n.summary()
    cnn = cnn_n.fit(x_train, y_train, batch_size=batch_size, epochs=epochz, validation_data=(x_test,y_test),shuffle=True)
    cnn_n.save_weights(model_name, overwrite=True)

    scores = cnn_n.evaluate(x_test, y_test, verbose=0)
    print("Acurácia: %.2f%%" % (scores[1] * 100))

#se tiver argumento, faz predição do diretorio de imagem passado como parametro
else:
    if not os.path.isdir(sys.argv[1]):
        print("Diretório não encontrado. Saindo...")
        exit(0)

    #faz loop nos arquivos e limpa os nomes
    directory = sys.argv[1]
    files     = os.listdir(directory)

    files = [w.replace("\n", "") for w in files]
    #print(files)

    #se for diretório vazio, sai sem dar erro
    if len(files) < 1:
        print("Não existe arquivos no diretório: \'"+directory+"\'. Saindo...")
        exit(0)

    #se tiver arquivos, faz a predição
    cnn_n = base_model()
    cnn_n.load_weights(model_name)

    font = cv2.FONT_HERSHEY_SIMPLEX

    #lê imagens de um diretório e redimensiona para o padrão do MNIST
    for filename in files:
        print(filename)
        img = cv2.imread(directory + "/" +filename)
        img = cv2.resize(img, (28, 28), 1)

        xxx = img_to_array(img)
        #print(str(xxx.shape))
        xxx = np.expand_dims(xxx, axis=0)
        #print(str(xxx))

        start_t = time.time()

        res = cnn_n.predict(xxx, verbose=0)

        print("res.argmax e res 0 argmax")
        print(res.argmax())
        print(res[0][res.argmax()])
        #print(res[0])

        detected = res.argmax()

        print(round(res[0][res.argmax()]*100))

        imgRead = cv2.imread(directory + "/" + filename, cv2.IMREAD_GRAYSCALE)
        cv2.putText(imgRead, str(res.argmax()), (2, 22), font, 1, (100, 100, 100), 2, cv2.LINE_AA)

        cv2.imshow("MINIST", imgRead)
        pressed = cv2.waitKey(0)
        if pressed == 27:
            exit(0)
        cv2.destroyAllWindows()

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Não foi possível encontrar o módulo especificado.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/er

<h3>Referências:</h3><br>

* https://www.cetax.com.br/blog/machine-learning/

* https://www.dobitaobyte.com.br/como-fazer-predicao-com-mnist-usando-arquivos-externos/

* https://penseallen.github.io/PensePython2e/

* https://github.com/wxs/keras-mnist-tutorial/blob/master/MNIST%20in%20Keras.ipynb

* https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py

* https://medium.com/data-hackers/entendendo-o-que-%C3%A9-matriz-de-confus%C3%A3o-com-python-114e683ec509